In [25]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
#!pip install dask[dataframe]
#!pip install pandas --upgrade
# We have to do this as dask is bad at guessing the types of objects for some reason
# Also, if you are having trouble with running the dataframe and continuously get 
# Fatal Python error: GC object already tracked, try to update pandas
data_types = {
    'ID':  np.int64,
    'Case Number':  object,
    'Date': object,
    'Block': object,
    'IUCR': object,
    'Primary Type': object,
    'Description':  object,
    'Location Description': object,
    'Arrest':  bool,
    'Domestic': bool,
    'Beat': np.int64,
    'District': np.float64,
    'Ward': np.float64,
    'Community Area': np.float64,
    'FBI Code': object,
    'X Coordinate': np.float64,
    'Y Coordinate': np.float64,
    'Year': np.int64,
    'Updated On': object,
    'Latitude': np.float64,
    'Longitude': np.float64,
    'Location': object
}
df = pd.read_csv('crimes.csv')

# get subset of crimes -- with over ~100 entries, the geolocator.geocode() call times out
crimes_sub  = df[:50]
crimes_sub['Arrest'] = crimes_sub['Arrest'].astype(int)
crimes_sub['Domestic'] = crimes_sub['Domestic'].astype(int)
crimes_sub.head()

/Users/kelseyball/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kelseyball/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,0,1,...,7,43,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
1,10446329,HZ185174,3/12/2016 23:50:00,034XX N HALSTED ST,460,BATTERY,SIMPLE,STREET,0,0,...,44,6,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
2,10446859,HZ185472,3/12/2016 23:50:00,006XX S CLARK ST,460,BATTERY,SIMPLE,RESIDENCE,0,0,...,2,32,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
3,10446323,HZ185185,3/12/2016 23:50:00,078XX S ABERDEEN ST,497,BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,APARTMENT,1,1,...,17,71,04B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
4,10446385,HZ185156,3/12/2016 23:45:00,068XX S JUSTINE ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,0,0,...,17,67,3,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN


In [31]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

# Gets full address using block address with "Chicago" appended
# Extracts zip code from full address
#
# TODO: address hyphenated zip code 60609-2524 parsed as two zip codes
#
def getZip(row):
    location = geolocator.geocode(row['Block'] + " Chicago")
    if location is None: 
        return 0
    address = location.address.replace(',', "")
    return next(iter([int(s) for s in address.split() if s.isdigit()]), 0)

# Add zip code column with zip code of crime
crimes_sub['Zip Code'] = crimes_sub.apply(getZip, axis=1, reduce=False)
crimes_sub.shape

/Users/kelseyball/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(50, 23)

In [32]:
crimes_sub.shape

(50, 23)

In [33]:
poverty = pd.read_csv('poverty.csv')
poverty.head()


,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6


In [34]:
# Socioeconomic stats by community area 
# https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2
#
poverty = pd.read_csv('poverty.csv')
poverty.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6


In [35]:
# Mapping of community area numbers to zip codes
# https://docs.google.com/spreadsheets/d/1YzfGnfeiLdYtbN8IoOctPZPJRCAms_gW6g-OTvkGAZU/edit#gid=0
#

zip_codes = pd.read_csv('zipcodes.csv')
zip_codes.head()

,Community Area Number,Zip Code
0,1,60626
1,1,60645
2,1,60660
3,2,60626
4,2,60645


In [53]:
# Merge socioeconomic stats with zip codes
poverty_by_ca = pd.merge(poverty, zip_codes, how='left', on='Community Area Number')

# get avg stats for zip code (each zip code includes multiple community areas)
# methodology not ideal -- obscures variance in community areas, takes mean of community area number (could fix by splitting table)
# --> granularity/scope of location data = challenge to talk about in presentation
poverty_by_zip = poverty_by_ca.groupby('Zip Code').mean()
poverty_by_zip.head()

,Community Area Number,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
Zip Code,,,,,,,,
60068,9,1.1,3.3,6.5,7.4,35.3,40959,8
60601,32,1.5,14.7,5.7,3.1,13.5,65526,3
60602,32,1.5,14.7,5.7,3.1,13.5,65526,3
60603,32,1.5,14.7,5.7,3.1,13.5,65526,3
60604,32,1.5,14.7,5.7,3.1,13.5,65526,3


In [37]:
# Merge crimes with socioeconomic data for relevant zip code
crimes_with_poverty = pd.merge(crimes_sub, poverty_by_zip, how='left', right_index=True, left_on='Zip Code')
crimes_with_poverty.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Location,Zip Code,Community Area Number,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,0,1,...,NaN,60617,49.50,3.716667,20.666667,16.466667,20.050000,41.600000,20731.000000,54.500000
1,10446329,HZ185174,3/12/2016 23:50:00,034XX N HALSTED ST,460,BATTERY,SIMPLE,STREET,0,0,...,NaN,60642,15.25,2.050000,14.175000,6.725000,8.450000,23.000000,58831.500000,9.000000
2,10446859,HZ185472,3/12/2016 23:50:00,006XX S CLARK ST,460,BATTERY,SIMPLE,RESIDENCE,0,0,...,NaN,60605,32.50,1.400000,14.250000,5.300000,5.250000,17.650000,62301.500000,5.000000
3,10446323,HZ185185,3/12/2016 23:50:00,078XX S ABERDEEN ST,497,BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,APARTMENT,1,1,...,NaN,60608,35.40,8.640000,27.780000,15.380000,33.120000,33.380000,20104.600000,67.000000
4,10446385,HZ185156,3/12/2016 23:45:00,068XX S JUSTINE ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,0,0,...,NaN,60609,43.00,5.885714,33.514286,21.285714,27.814286,38.728571,16810.142857,73.714286


In [50]:
# Get number of crimes in each zip code
crimes_by_zip = crimes_with_poverty.groupby('Zip Code').ID.count()
crimes_by_zip.head()

Zip Code
0        3
2524     1
60038    1
60603    1
60605    1
Name: ID, dtype: int64

In [51]:
# Get number of arrests in each zip code
arrests_by_zip = crimes_with_poverty.groupby('Zip Code').Arrest.sum()
arrests_by_zip.head()

Zip Code
0        1
2524     0
60038    0
60603    1
60605    0
Name: Arrest, dtype: int64